In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
import torch
import numpy as np
import torch.nn.functional as F
import torch.nn as nn

import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torch.utils.data.dataset import Dataset, Subset
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import cv2
import copy


from torch._utils import _accumulate

In [11]:
class ObjectsDataset(Dataset):
  
  def __init__(self, dataset_dir, frames, labels_index, labels_names, image_type):
    # image_type can be original, gray image or color-opponency. It is expected 
    # that images with these types are already available - gotten through preprocessing offline. 
    
    self.labels = labels_index
    self.dataset_dir = dataset_dir
    self.transforms = transforms
    self.image_type = image_type
    self.frames = frames
    self.labels_names = labels_names
    self.dataset_dir = dataset_dir
    
    
  def __len__(self):
    return len(self.frames)
    
  def __getitem__(self, index):
    frame_path = self.frames[index]
    label_index = self.labels[index]
    plant_name = self.labels_names[label_index]
    
    image = cv2.imread(frame_path)
    #image = cv2.resize(image, (1080, 720))  
    
    image = image/ 255.0
    image=torch.from_numpy(image.astype('float32')).permute(2, 0, 1)
    
    label=torch.from_numpy(np.asarray(label_index).astype('long'))
        
    return (image, label) 

In [24]:
import os
import re

def natural_key(string_):
    """See http://www.codinghorror.com/blog/archives/001018.html"""
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string_)]

def get_dataloader(image_type='original', skip=6, batch_size=64, shuffle=True):
    classes = ['bag','beer','book','case','coffee','cup','deodorant','eraser',
               'hole','mouse','mug','sleep','speaker','spray','stapler','tape',
               'tea','tissues','umbrella','watch']
    
    root_path = '/content/drive/My Drive/RODframes2/RODframes/'
    model_path = '/content/drive/My Drive/Colab Notebooks/RetinaSmartCamera/models/'

    #image_path = os.path.join(root_path)
    train_path = os.path.join(root_path, 'train/')
    test_path = os.path.join(root_path, 'test/')
    
    master_frames = []
    master_labels = []
    
    frames = []
    labels = []

    errors = []
    min_count = 7200
    for index, obj in enumerate(classes):
        print('Iterating over {}'.format(obj))
        try:
            object_frames = []
            labels_index = []

            object_path = os.path.join(train_path, obj+'/light/original/')
            count_index = 0

            for frame_index, frame in enumerate(os.listdir(object_path)):
                if count_index%skip == 0:
                    frame_path = os.path.join(object_path, frame)
                    object_frames.append(frame_path)
                    labels_index.append(frame_index)
                count_index += 1
            
            if len(object_frames) < min_count:
                min_count = len(object_frames)
            sorted_object_frames = sorted(object_frames, key=natural_key)
            master_frames.append(sorted_object_frames)
            master_labels.append(labels_index)
            print('Number of images for class ',obj, ': ', len(sorted_object_frames))
        
        except OSError as err:
            print("OS error for object {}: {}".format(obj, err))
            errors.append((index, obj))
            continue
    
    for ind, obj in errors:
        classes.remove(obj)
        del master_frames[ind]
        del master_labels[ind]
    print(classes)
    
    for i in range(0, min_count):
        for object_index in range(len(classes)):
            frames.append(master_frames[object_index][i])
            labels.append(master_labels[object_index][i])

    dataset = ObjectsDataset(root_path, frames, labels, classes, image_type)

    train_size = int (0.8 * len(dataset))
    validation_size = int(0.15 * len(dataset))
    test_size = len(dataset) - train_size - validation_size
    train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, validation_size, test_size])

    dataset_dict = {'train' : train_dataset, 'validation' : validation_dataset, 'test' : test_dataset}

    dataloader = {x : torch.utils.data.DataLoader(dataset_dict[x], batch_size = batch_size, shuffle = shuffle, 
                                                  num_workers = 0, drop_last=True) for x in ['train', 'test', 'validation']}
    
    dataset_sizes = {x: len(dataset_dict[x]) for x in ['train', 'test', 'validation']}
    print('Dataset size is ', dataset_sizes)

    return dataloader
    

In [59]:
def createLabels(classes, set_, root_dir, light,mode,bg=False):
  
    labels=[]
    frame=[]
    lightt=[]
    k=0
    if bg==True:
      k=20
    for i in classes:
        frames = os.listdir(os.path.join(root_dir, set_, i,light,mode))
        for a in frames:
            labels.append(k)
            frame.append(a)
            lightt.append(light)
        k+=1
    return labels,frame, lightt
  
def orderFramesSequence(labels,light,c=3):
  
  frame=[]
  lab=[]
  lt=[]
  rang=[]
  label,counts=np.unique(labels,return_counts=True)

  for h in range(c):
    rang.append(counts.copy())
  
  for a in label:
    
    f=counts[a].copy()
    for v in range(c):
      
      rem=(f-v) % c
      val=f-rem-v
      rang[v][a] = val
      
      
    for i in range(c):
      
      for x in range(rang[i][a]):
        
        if ((x+1+i)%c)==(i+1):
          frame.append('frame'+str(x+1+i)+'.png')
          lab.append(a)
          lt.append(light)
          
        elif c==(i+1):
          if ((x+1+i)%c)==0:         
            frame.append('frame'+str(x+1+i)+'.png')
            lab.append(a)
            lt.append(light)
        
  return frame,lab,lt


def random_split(dataset, lengths,indic=True,indices=[]):
    """
    Randomly split a dataset into non-overlapping new datasets of given lengths.

    Arguments:
        dataset (Dataset): Dataset to be split
        lengths (sequence): lengths of splits to be produced
    """
    if sum(lengths) != len(dataset):
        raise ValueError("Sum of input lengths does not equal the length of the input dataset!")
    if indic==True:
      indices = randperm(sum(lengths))
    return [Subset(dataset, indices[offset - length:offset]) for offset, length in zip(_accumulate(lengths), lengths)]

class MyDataset(Dataset):
 

    def __init__(self,classes, labels, light, root_dir,sett,typee,frames,transform=None,c=3,rgb=False):
       
        self.labels = labels
        self.root_dir = root_dir
        self.sett=sett
        self.transform = transform
        self.typee=typee
        self.frame=frames
        self.classes=classes
        self.light=light
        self.c=c
        self.rgb=rgb

    def __len__(self):
        return len(self.frame)
      

    
    def __getitem__(self, ind):
        img_name = os.path.join(self.root_dir,self.sett, 
                                self.classes[self.labels[ind]],self.light[ind],self.typee,self.frame[ind])
        image = cv2.imread(img_name)
        if self.rgb==False:
            image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        image=image/255
        
        frame_=self.frame[ind][5::]
        ind_=int(frame_.replace('.png',''))+1
        images=image
        
        for g in range(self.c-1):
          frame_='frame'+str(ind_+g)+'.png'
        
          img_name2 = os.path.join(self.root_dir,self.sett, 
                                  self.classes[self.labels[ind]],self.light[ind],self.typee,frame_)
          image2 = cv2.imread(img_name2)
          if rgb==False:
              image2=cv2.cvtColor(image2,cv2.COLOR_BGR2GRAY)
          image2=image2/255
          images=np.concatenate((images,image2)) 
        
        label = self.labels[ind]
        sample = (images , label)

        if self.transform:
            sample = self.transform(sample)

        return sample

class ToTensor(object):
    

    def __call__(self, sample):
        image, label = sample
        
        

        image=torch.from_numpy(image.astype('float32')).view(-1,257,490)
        
        label=torch.from_numpy(np.asarray(label).astype('long'))
        
        return (image , label)


def createDataloaders(root_dir, classes, bs, c, set_, type_, testset_, train_ind, val_ratio, color_opponency):
  while True:
    try:
      labels_light, frame_light,lightt = createLabels(classes, 'train', root_dir, 'light',set_+'_'+type_)
      #labels_dark, frame_dark, darkk = createLabels(classes, 'train', root_dir, 'dark',set_+'_'+type_)
      if c>1:
        frame_light, labels_light, lightt = orderFramesSequence(labels_light, 'light',c)
        #frame_dark, labels_dark, darkk =orderFramesSequence(labels_dark, 'dark',c)
      labels=labels_light#+labels_dark
      frame=frame_light#+frame_dark
      light=lightt#+darkk
      
      if color_opponency==False:
          train = MyDataset(classes, labels, light, root_dir,'train',set_+'_'+type_,frame, transforms.Compose([ ToTensor()]),c=c)
      else:
          train = MyDatasetColourOpponency(classes, labels, light, root_dir,'train',set_+'_'+type_,frame, transforms.Compose([ ToTensor()]),c=c)




      leng = len(train)
      test_len = int(leng * val_ratio)
      train_len = leng - test_len
      indices=train_ind
      val_set, train_set_ = random_split(train, [test_len, train_len],True,torch.from_numpy(indices))


  #------------------------------------------------------------------------------------------------------------------------------------------------

      train_dl = DataLoader(train_set_, batch_size=bs,shuffle=True, num_workers=15)

      val_dl = DataLoader(val_set, batch_size=bs,shuffle=True, num_workers=15)

      labels_light, frame_light,lightt = createLabels(classes, 'test', root_dir, 'light',testset_+'_'+type_)
      if c>1:
         frame_light, labels_light, lightt = orderFramesSequence(labels_light, 'light',c=c)
      if testset_=='original':
          pass
          #labels_dark, frame_dark, darkk = createLabels(classes, 'test', root_dir, 'dark',testset_+'_'+type_)
      if c>1:
          #frame_dark, labels_dark, darkk =orderFramesSequence(labels_dark, 'dark',c=c)
        
          labels=labels_light#+labels_dark
          frame=frame_light#+frame_dark
          light=lightt#+darkk
      else:
          labels=labels_light
          frame=frame_light
          light=lightt
      if color_opponency==False:
          test_set = MyDataset(classes, labels, light, root_dir,'test',testset_+'_'+type_,frame, transforms.Compose([ ToTensor()]),c=c)
      else:
          test_set = MyDatasetColourOpponency(classes, labels, light, root_dir,'test',testset_+'_'+type_,frame, transforms.Compose([ ToTensor()]),c=c)
    
      test_dl = DataLoader(test_set, batch_size=bs, num_workers=10,shuffle=False)
        
    
      break
    except OSError as err:
      print("OS error: {0}".format(err))
      continue
    
  dataloaders = {'train': train_dl, 'val': val_dl, 'test': test_dl}
  return dataloaders

In [60]:
from torch import randperm
root_dir = '/content/drive/My Drive/RODframes2/RODframes/'
classes = ['bag','beer','book','case','coffee','cup','deodorant','eraser',
               'hole','mouse','mug','sleep','speaker','spray','stapler','tape',
               'tea','tissues','umbrella','watch']
bs=50
n_frames=1
set_='original'
type_='corticalimages'
testset_ = 'original'
color_opponency=False
val_ratio = 0.2
train_ind = np.array([])
dataloaders = createDataloaders(root_dir, classes, bs, n_frames, set_, type_, testset_, train_ind, val_ratio, color_opponency)


In [43]:
dataset = get_dataloader(image_type = '', skip = 10, batch_size=8)

Iterating over bag
Number of images for class  bag :  210
Iterating over beer
Number of images for class  beer :  217
Iterating over book
Number of images for class  book :  212
Iterating over case
Number of images for class  case :  197
Iterating over coffee
Number of images for class  coffee :  209
Iterating over cup
Number of images for class  cup :  224
Iterating over deodorant
Number of images for class  deodorant :  194
Iterating over eraser
Number of images for class  eraser :  212
Iterating over hole
Number of images for class  hole :  212
Iterating over mouse
Number of images for class  mouse :  201
Iterating over mug
Number of images for class  mug :  210
Iterating over sleep
Number of images for class  sleep :  209
Iterating over speaker
Number of images for class  speaker :  205
Iterating over spray
Number of images for class  spray :  204
Iterating over stapler
Number of images for class  stapler :  208
Iterating over tape
Number of images for class  tape :  200
Iterating 

In [66]:
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        ## encoder layers ##
        # conv layer (depth from 3 --> 16), 3x3 kernels
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)  
        # conv layer (depth from 16 --> 4), 3x3 kernels
        self.conv2 = nn.Conv2d(16, 4, 3, padding=1)
        # pooling layer to reduce x-y dims by two; kernel and stride of 2
        self.pool = nn.MaxPool2d(2, 2)
        
        ## decoder layers ##
        ## a kernel of 2 and a stride of 2 will increase the spatial dims by 2
        self.t_conv1 = nn.ConvTranspose2d(4, 16, 2, stride=2)
        self.t_conv2 = nn.ConvTranspose2d(16, 3, 2, stride=2)

    def forward(self, x):
        ## encode ##
        # add hidden layers with relu activation function
        # and maxpooling after
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        # add second hidden layer
        x = F.relu(self.conv2(x))
        x = self.pool(x)  # compressed representation
        
        ## decode ##
        # add transpose conv layers, with relu activation function
        x = F.relu(self.t_conv1(x))
        # output layer (with sigmoid for scaling from 0 to 1)
        x = self.t_conv2(x)
        x = F.sigmoid(x)
                
        return x
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=(5,5))
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2,2), return_indices=True)
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2,2), return_indices=True)
        self.unconv1 = nn.ConvTranspose2d(6,3,kernel_size=(5,5))
        self.maxunpool1 = nn.MaxUnpool2d(kernel_size=(2,2))
        self.unmaxunpool2 = nn.MaxUnpool2d(kernel_size=(2,2))
        
        self.encoder1 = nn.Sequential(
            nn.Tanh(),
            nn.Conv2d(6, 12,kernel_size=(5,5)),
        )
        
        self.encoder2 = nn.Sequential(
            nn.Tanh(),
            nn.Conv2d(12, 16, kernel_size=(5,5)),
            nn.Tanh()
        )
        
        self.decoder2 = nn.Sequential(
            nn.ConvTranspose2d(16, 12, kernel_size=(5,5)),
            nn.Tanh()
        )
        
        self.decoder1 = nn.Sequential(
            nn.ConvTranspose2d(12,6,kernel_size=(5,5)),
            nn.Tanh(),
        )
        

    def forward(self, x):
        # Encoder
        x = self.conv1(x)
        x,indices1 = self.maxpool1(x)
        x = self.encoder1(x)
        x,indices2 = self.maxpool2(x)
        x = self.encoder2(x)
        
        # Decoder
        x = self.decoder2(x)
        x = self.unmaxunpool2(x, indices2)
        x = self.decoder1(x)
        x = self.maxunpool1(x,indices1)
        x = self.unconv1(x)
        x = nn.Tanh()(x)
        return x


In [67]:
model = ConvAutoencoder()
#model = Autoencoder()

criterion = nn.CrossEntropyLoss()

# specify loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
n_epochs = 20

for epoch in range(1, n_epochs+1):
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    for data, label in dataloaders['train']:
       
        # _ stands in for labels, here
        # no need to flatten images
        images = data#, _ = data
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        print(images.shape)
        outputs = model(images)
        # calculate the loss
        loss = criterion(outputs, images)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*images.size(0)
            
    # print avg training statistics 
    train_loss = train_loss/len(train_loader)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch, 
        train_loss
        ))

torch.Size([50, 1, 257, 490])


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


RuntimeError: ignored